In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [14]:
import csv
import numpy as np

def read_csv_into_2d_list(file_name):
    data_2d_list = []
    with open(file_name, newline='') as f:
        csvreader = csv.reader(f)
        for row in csvreader:
            data_2d_list.append(row)
    return data_2d_list

mnist_data = read_csv_into_2d_list("mnist_train.csv")

X = torch.zeros(len(mnist_data), 784, dtype=torch.float32)
Y = torch.zeros(len(mnist_data), 10, dtype=torch.float32)

for i in range(0, len(mnist_data)):
    for j in range(0, 784):
        X[i][j] = float(mnist_data[i][j+1]) / 255.0
    Y[i][int(float(mnist_data[i][0]))] = 1.0

In [15]:
X.shape, Y.shape

(torch.Size([50559, 784]), torch.Size([50559, 10]))

In [18]:
image = 10

plt.imshow(X[image].view(28,28), cmap='gray')
print(Y[image])

tensor([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.])
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 1., 0., 0., 0., 0., 0.])
tensor([0., 1., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])
tensor([0., 0., 1., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 1., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 1., 0., 0., 0., 0., 0., 0.])
tensor([0., 1., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 1., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 1., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.])
tensor([0., 0., 0., 1., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 1., 0., 0., 0.])
tensor([0., 1., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 1., 0., 0.])
tensor([0., 0., 1., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 1., 0.])
tensor([0., 0., 0., 0., 0., 0., 1., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])
tensor([0., 0., 0., 

KeyboardInterrupt: 

Error in callback <function _draw_all_if_interactive at 0x1658680e0> (for post_execute):


KeyboardInterrupt: 